<a href="https://colab.research.google.com/github/fipeop/ctci-solutions/blob/master/XRD-Colab-V2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import files
uploaded = files.upload()

Saving exp_train_x.csv to exp_train_x (1).csv
Saving exp_train_y.csv to exp_train_y (1).csv
Saving X_exp.csv to X_exp (1).csv
Saving X_th.csv to X_th (1).csv
Saving y_exp.csv to y_exp.csv
Saving y_th.csv to y_th.csv


Let's see what this file looks like.

In [8]:
#Call to bunch of libraries and functions, just run 

import time  
from sklearn import metrics
import numpy as np  
import matplotlib.pyplot as plt
from matplotlib import cm
import os
import pandas as pd
from scipy.signal import savgol_filter
from scipy.signal import find_peaks_cwt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Model
import keras as K
#from keras models import Sequential
from keras.models import Sequential
from keras.callbacks import ReduceLROnPlateau

##define multiple machine learning algorithms


import tensorflow as tf

K.backend.clear_session()

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

tf.reset_default_graph()
# Parameters

BATCH_SIZE=256

# Network Parameters
n_input = 1200 # MNIST data input (img shape: 28*28)
n_classes = 7 # MNIST total classes (0-9 digits)
#drop_out = 0.2 # Dropout, probability to keep units
filter_size = 2
kernel_size = 10

enc = OneHotEncoder(sparse=False)

#def convnet(X_in,n_classes):
#   
#       
#   

model = Sequential()

model.add(K.layers.Conv1D(32, 8, padding='same',input_shape=(1200,1), activation='relu'))
model.add(K.layers.Conv1D(32, 5, padding='same', activation='relu'))
model.add(K.layers.Conv1D(32, 3, padding='same', activation='relu'))
model.add(K.layers.pooling.GlobalAveragePooling1D())

print(model.summary())

#model.add(K.layers.Dense(128, activation='relu'))

model.add(K.layers.Dense(n_classes, activation='softmax'))


#data normalization from 0 to 1 for double column dataframe, returns single column array
def normdata(data):
    (len1,w1) = np.shape(data)
    ndata = np.zeros([len1,w1//2])
    for i in range(w1//2):
        ndata[:,i]=(data[:,2*i+1]-min(data[:,2*i+1]))/(max(data[:,2*i+1])-min(data[:,2*i+1]))
    return ndata
#data normalization from 0 to 1 for single column dataframe
def normdatasingle(data):
    (len1,w1) = np.shape(data)
    ndata = np.zeros([len1,w1])
    for i in range(w1):
        ndata[:,i]=(data[:,i]-min(data[:,i]))/(max(data[:,i])-min(data[:,i]))
    return ndata

#data augmendatation for simulated XRD spectrum
def augdata(data,num,dframe,minn,maxn):
    (len1,w1) = np.shape(data)
    augd =np.zeros([len1,num])
    naugd=np.zeros([len1,num])
    newaugd=np.zeros([len1,num])
    crop_augd = np.zeros([maxn-minn,num])
    par1 = list(dframe.columns.values)
    pard = []
    for i in range(num):
        rnd = np.random.randint(0,w1)
        # create the first filter for peak elimination
        dumb= np.repeat(np.random.choice([0,1,1],300),len1//300)
        dumb1= np.append(dumb,np.zeros([len1-len(dumb),]))
        # create the second filter for peak scaling
        dumbrnd= np.repeat(np.random.rand(100,),len1//100)
        dumbrnd1=np.append(dumbrnd,np.zeros([len1-len(dumbrnd),]))
        #peak eleminsation and scaling
        augd[:,i] = np.multiply((data[:,rnd]),dumbrnd1)
        augd[:,i] = np.multiply(augd[:,i],dumb1)
        #nomrlization
        naugd[:,i] = (augd[:,i]-min(augd[:,i]))/(max(augd[:,i])-min(augd[:,i])+1e-9)
        pard.append (par1[2*rnd])
        #adding shift
        cut = np.random.randint(-20*1,20)
        #XRD spectrum shift to left
        if cut>=0:
            newaugd[:,i] = np.append(naugd[cut:,i],np.zeros([cut,]))
        #XRD spectrum shift to right
        else:
            newaugd[:,i] = np.append(naugd[0:len1+cut,i],np.zeros([cut*-1,]))
         
        crop_augd[:,i] = newaugd[minn:maxn,i]
#        
    return newaugd, pard,crop_augd
#data augmendatation for experimental XRD spectrum
def exp_augdata(data,num,label):
    (len1,w1) = np.shape(data)
    augd =np.zeros([len1,num])
    naugd=np.zeros([len1,num])
    newaugd=np.zeros([len1,num])
    par=np.zeros([num,])
    for i in range(num):
        rnd = np.random.randint(0,w1)

         # create the first filter for peak elimination
        dumb= np.repeat(np.random.choice([0,1,1],300),len1//300)
        dumb1= np.append(dumb,np.zeros([len1-len(dumb),]))
        # create the second filter for peak scaling
        dumbrnd= np.repeat(np.random.rand(200,),len1//200)
        dumbrnd1=np.append(dumbrnd,np.zeros([len1-len(dumbrnd),]))
        #peak eleminsation and scaling
        augd[:,i] = np.multiply((data[:,rnd]),dumbrnd1)
        augd[:,i] = np.multiply(augd[:,i],dumb1)
        #nomrlization
        naugd[:,i] = (augd[:,i]-min(augd[:,i]))/(max(augd[:,i])-min(augd[:,i])+1e-9)
        par[i,] =label[rnd,] 
        #adding shift
        cut = np.random.randint(-20*1,20)
        #XRD spectrum shift to left
        if cut>=0:
            newaugd[:,i] = np.append(naugd[cut:,i],np.zeros([cut,]))
        #XRD spectrum shift to right
        else:
            newaugd[:,i] = np.append(naugd[0:len1+cut,i],np.zeros([cut*-1,]))

    return newaugd, par

#extracting exprimental data
def exp_data_processing (data,minn,maxn,window):
    (len1,w1) = np.shape(data)
    nexp1 =np.zeros([maxn-minn,w1])
    for i in range(w1):
        #savgol_filter to smooth the data
         new1 = savgol_filter(data[minn:maxn,i], 31, 3)
         #peak finding
         zf= find_peaks_cwt(new1, np.arange(10,15), noise_perc=0.01)
         #background substraction
         for j in range(len(zf)-1):
             zf_start= np.maximum(0,zf[j+1]-window//2)
             zf_end = np.minimum(zf[j+1]+window//2,maxn)
             peak = new1[zf_start:zf_end]
             
             ##abritaryly remove 1/4 data
             npeak = np.maximum(0,peak-max(np.partition(peak,window//5 )[0:window//5]))
             nexp1[zf_start:zf_end,i]= npeak     
    return nexp1

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1200, 32)          288       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1200, 32)          5152      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1200, 32)          3104      
_________________________________________________________________
global_average_pooling1d_1 ( (None, 32)                0         
Total params: 8,544
Trainable params: 8,544
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
X_th=pd.read_csv("X_th.csv", index_col='Unnamed: 0')
y_th=pd.read_csv("y_th.csv", index_col='Unnamed: 0')
y_exp=pd.read_csv("y_exp.csv", index_col='Unnamed: 0')
X_exp=pd.read_csv("X_exp.csv", index_col='Unnamed: 0')

exp_train_x=pd.read_csv("exp_train_x.csv", index_col='Unnamed: 0')
exp_train_y=pd.read_csv("exp_train_y.csv", index_col='Unnamed: 0')

#X_exp.shape
#y_exp.shape
#y_th.shape
X_th.shape

(772, 1200)

In [14]:
#data augmentation to experimenal traning dataset
#exp_train_x,exp_train_y = exp_augdata(temp_x.T,2000,temp_y)
#combine theorectical and experimenal dataset for training
train_combine = np.concatenate((X_th,exp_train_x.T))
  
train_dim = train_combine.reshape(train_combine.shape[0],1200,1)
train_y = np.concatenate((y_th,exp_train_y))
train_y_hot = enc.fit_transform(train_y.reshape(-1,1))


#sess = tf.Session ()
#sess.run(tf.global_variables_initializer())

      
optimizer = K.optimizers.Adam()

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                              patience=50, min_lr=0.00001)




hist = model.fit(train_dim, train_y_hot, batch_size=BATCH_SIZE, nb_epoch=10000,
                 verbose=1, callbacks = [reduce_lr])










/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


ValueError: ignored

In [0]:
from keras.models import load_model
model = load_model('keras_model.h5')        
from vis.visualization import visualize_cam

import matplotlib.pyplot as plt
import numpy as np

#ind = np.where(bbb==1)[0]
#for j,modifier in enumerate([None, 'guided', 'relu']):
#    heat_map = visualize_cam(model, 3, bbb[ind],train_dim[ind], backprop_modifier=modifier)
#
#    plt.imshow(heat_map)
#plt.imshow(heat_map)




# get the weights from the last layer
gap_weights = model.layers[-1].get_weights()[0]

# create a new model to output the feature maps and the predicted labels
cam_model = Model(inputs=model.input, 
                    outputs=(model.layers[-4].output, model.layers[-1].output)) 

# make the prediction for a set of test images
features, results = cam_model.predict(test_x)

# check the prediction for 10 test images
for idx in range(7):   
    # get the feature map of the test image
    features_for_one_img = features[idx, :, :]
    

    # map the feature map to the original size

    cam_features = features_for_one_img
        
    # get the predicted label with the maximum probability
    pred = np.argmax(results[idx])
    
    # prepare the final display
    plt.figure(facecolor='white')
    
    # get the weights of class activation map
    cam_weights = gap_weights[:, pred]

    # create the class activation map
    cam_output = np.dot(cam_features, cam_weights)
    
    x = np.linspace(10,60,1200)
    y = abs(cam_output)
    
    fig, (ax,ax2) = plt.subplots(nrows=2, sharex=True)
    
    extent = [x[0]-(x[1]-x[0])/2., x[-1]+(x[1]-x[0])/2.,0,1]
    ax.imshow(y[np.newaxis,:], cmap="plasma", aspect="auto", extent=extent)
    ax.set_yticks([])
    ax.set_xlim(extent[0], extent[1])
    
    ax2.plot(x,y)
    
    plt.tight_layout()
    plt.show()
    
    ax2.plot(x,y)
    
    plt.tight_layout()
    plt.show() 